In [1]:
#Master file for creation of CASTEP inputs as well as running calculations

import pickle
import glob
import subprocess
import shutil
import os

import ase
from ase import io

import soprano
from soprano.properties.nmr import *

import dscribe
from dscribe.descriptors import SOAP

import numpy as np

In [4]:
#Overall could add some module levels blocks e.g. for uid dict loading and general pickle loading /dumping

   #MAIN
#c2x
def ciffer():

    def cif2cell():
        raw_cifs = glob.glob('Cifs/Raw_Cifs/*.cif')

        for i in raw_cifs:

            process = subprocess.run('cif2cell ' + i + ' --program=castep -o ' + i[:-4] + '.cell', shell=True,
                                     stdout=subprocess.PIPE) #run cif2cell
            if process.returncode == 0:
                print('cif2cell succesful for ' + i)
                shutil.move(i, 'Cifs/Backup/')
                if os.path.exists(i[:-4] + '.cell'):
                    shutil.move(i[:-4] + '.cell', 'Cells/Raw Cells/') #move to raw cells file
                else:
                    print('no .cell found for ' + i)
            elif process.returncode != 0:
                print('cif2cell failed for ' + i)

    def c2x_cif():
        raw_cifs = glob.glob('Cifs/Raw_Cifs/*.cif')

        for i in raw_cifs:
            process = subprocess.run('c2x --cell '  + i + ' '+ i[:-4] + '.cell', shell=True,
                                     stdout=subprocess.PIPE) #run c2x
            if process.returncode == 0:
                print('c2x succesful for ' + i ) 
                shutil.move(i, 'Cifs/Backup/')
                if os.path.exists(i[:-4] + '.cell'):
                    shutil.move(i[:-4] + '.cell', 'Cells/Raw Cells/') #move to raw cells file
                else:
                    print('no .cell found for ' + i)
            elif process.returncode != 0:
                print('c2x failed for ' + i)
                os.remove(i[:-4] + '.cell')
                
    cif2cell()
    c2x_cif()
    
    return print('ciffing complete')

def c2x_pos():
    return
    

        

#Reference dictionary / UID
#creates a dictionary with reference label and moves reindexed structure to main file + creates backup
def cell_index():
    raw_cells = glob.glob('Cells/Raw Cells/*')
    filtered = []
    
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        uid_dict = {} #otherwise initalise new dic
        uid_dict['1'] = 'ref' 
    
    for i in raw_cells:
        
        #check if file is already in backup hence being a cuplicate
        if os.path.exists('Cells/Backup_Cells/' + i[16:]):
            print(i + ' is a dupicate skipping')
            os.remove(i)
        
        #else adding to dictionary
        else:  
            keys = list(uid_dict.keys())
            uid_dict[int(keys[-1])+1] = i[16:-5] # just use file name which should be formatted.
            #  input('ref for ' + str(i) + ' :') # this needs improvement maybe add a nested dictionary key for refs
            #if all additons are from same source could hardcode a value.
            shutil.copyfile(i, 'Cells/UID Cells/' + str(int(keys[-1])+1) + '.cell')
            shutil.move(i, 'Cells/Backup_Cells/')
        
    pickle.dump(uid_dict,open('uid_index.pkl','wb')) #saving data structure as binary pickle object
    return uid_dict  


#makes a directory for all uids in dic without one
#will print if no .cell found
def dir_maker():
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        print('no dic found')
        return
    
    #checks which directories have been made and which need to be made returning a list of UIDS to be made
    def keys_to_do(uid_dict):
        keys = list(uid_dict.keys())
        keys.remove('1')
        keys_to_do = []

        def filterer(keys):
            if os.path.exists('Main_Data/' + str(keys)):
                return False
            else:
                return True

        filtered = filter(filterer,keys)
        for i in filtered:
            keys_to_do.append(i)

        return keys_to_do
    
    keys_to_do = keys_to_do(uid_dict)
    
    for i in keys_to_do:
        os.mkdir('Main_Data/' + str(i))
        if os.path.exists('Cells/UID Cells/' + str(i) + '.cell'):
            shutil.copyfile('Cells/UID Cells/' + str(i) + '.cell', 'Main_Data/' + str(i) + '/' + str(i) + '.cell')
        else:
            print('no .cell found for :' + str(i))
            os.rmdir('Main_Data/' + str(i))
            
            
#updating .cell file with k spcae line or creating line
def cell_corrector(k_space):    #write the .cell file with correct params from a cif2cell file
    
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        print('no dic found')
        return    

    keys = list(uid_dict.keys())
    keys.remove('1')
    
    for i in keys:


        cell = open('Main_Data/'+ str(i)+'/'+str(i)+'.cell', 'r')
        contents = cell.read()
        lines = contents.splitlines(True)


        if 'kpoints_mp_spacing' in contents: #check if already has a kpoints line
            for l in lines:  
                if 'kpoints_mp_spacing' in l:  
                    lines[lines.index(l)] = 'kpoints_mp_spacing ' + str(k_space) + '\n'
                    cell = open(f'Main_Data/'+ str(i)+'/'+str(i)+'.cell', 'w')
                    cell.writelines(lines)
                    cell.close()
        else:
            cell = open('Main_Data/'+ str(i)+'/'+str(i)+'.cell', 'a')
            cell.write('\nkpoints_mp_spacing ' + str(k_space) + '\n')
            cell.close()

            
#write or overwrite the param file for all structures.
def param_writer(e_cut, xc_functional):
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        print('no dic found')
        return    

    keys = list(uid_dict.keys())
    keys.remove('1')
        
    for i in keys:
        param = open('Main_Data/'+ str(i)+'/'+str(i)+'.param', 'w')
        param.write('cut_off_energy ' + str(e_cut) + ' eV' + '\n'
                   + 'xc_functional ' + xc_functional + '\n'
                   + 'opt_strategy speed' + '\n'
                   + 'task magres' + '\n'
                   + 'magres_task nmr' + '\n'
                   + 'grid_scale 2' + '\n'
                   + 'data_distribution gvector' + '\n'
                   )
        param.close()
        
#dry runner to check if everything works
def dry_runner():

    def runner(to_run): #to_run will be a list of UIDs we want to run
        for i in to_run:
            subprocess.run('castep.serial -d Main_Data/' + str(i) + 
                           '/' + str(i), shell=True)
            
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        return  print('no dic found')
    
    keys = list(uid_dict.keys())
    keys.remove('1')
    
    runner(keys)
    
    for i in keys:
        if os.path.exists('Main_Data/' +str(i) + '/'+str(i)+ '.0001.err'):
            print('running error for ' +str(i))
            os.remove('Main_Data/' +str(i) + '/'+str(i)+ '.0001.err')
            
            uid_dict.pop(i) #removeing value from dic that fail
            #good deletre tree as well if wanted
        
        os.remove('Main_Data/'+str(i)+'/'+str(i)+'.castep')
        
    os.rename('uid_index.pkl', 'uid_index_original.pkl')    
    pickle.dump(uid_dict, open('uid_index.pkl','wb'))

        
        
    

#function to run castep has various modes see below
#maybe add a overwrite mode that clear previous before
#this need updating if later .magres files are moved..
#maybe add some other stuff printing progress etc
def castep_runner(mode, cores):
    
    def runner(to_run, cores): #to_run will be a list of UIDs we want to run
        for i in to_run:
            subprocess.run('mpirun -np ' + str(cores) + ' castep.mpi Main_Data/' + str(i) + 
                           '/' + str(i), shell=True)
            
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        return  print('no dic found')
    
    keys = list(uid_dict.keys())
    keys.remove('1')
    
    if mode == 0: #run all possible
        to_run = keys
        
        runner(to_run, cores)
              
    if mode == 1: #run range of all possible
        s_uid = int(input('State the start of the uid range'))
        f_uid = int(input('State the end of the uid range'))
        to_run = range(s_uid, f_uid+1, 1)
        
        runner(to_run, cores)
        
        
    if mode == 2: #check which files have not been run and run all of these.
        to_run = []
        for i in keys:
            if not os.path.exists('Main_Data/' + str(i) + '/' + str(i) + '.magres'):
                to_run.append(i)
        runner(to_run, cores)
        
    if mode == 3: #check which files have been run and run a numer of these
        to_run = []
        for i in keys:
            if not os.path.exists('Main_Data/' + str(i) + '/' + str(i) + '.magres'):
                to_run.append(i)
        num_run = int(input('how many structures do you want to run?'))
        test = to_run[:num_run]
        print(test)
        runner(test, cores)
        
    if mode > 3:
        return print('unsupported mode')
    
        
#copys all the magres files into the magres folder
#maybe add an option to overwrite or not
#maybe should go into a dated folder or something to allow for multiple copies
def magres_copyer():
                
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        return  print('no dic found')
    
    keys = list(uid_dict.keys())
    keys.remove('1')
    
    for i in keys:
        if os.path.exists('Main_Data/' + str(i) + '/' + str(i) + '.magres') and not os.path.exists('Main_Data/Magres/' + str(i) + '.magres'):
                shutil.copyfile('Main_Data/' + str(i) + '/' + str(i) + '.magres', 'Magres/' + str(i) + '.magres')

                
#creates a dic from magres for each structure in each file
#should add check if already exists and overwrite etc
def magres_cap():
    data = {}
    properties = ['iso','vzz','efg','fefg','mst','position','descriptor', 'kernel']
    
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        return  print('no dic found')
    
    keys = list(uid_dict.keys())
    keys.remove('1')
    
    for i in keys:
        if os.path.exists('Magres/' + str(i) + '.magres'):
        
            counter = -1

            magres = ase.io.read('Magres/' + str(i) + '.magres')
            atom_spec = magres.get_array('labels')
            indices = magres.get_array('indices')
            labels = ["{0}_{1}".format(l, i) for l, i in zip(atom_spec, indices)]

            iso = MSIsotropy.get(magres)
            vzz = EFGVzz.get(magres)
            efg = EFGAnisotropy.get(magres)
            fefg = magres.get_array('efg')
            mst = magres.get_array('ms')
            pos = magres.get_positions()


            #creating a empty nested dictioary to store data
            data = dict.fromkeys(labels)

            for l in labels:
                counter += 1

                data[l] = dict.fromkeys(properties)

                data[l][properties[0]] = iso[counter]
                data[l][properties[1]] = vzz[counter]
                data[l][properties[2]] = efg[counter]
                data[l][properties[3]] = fefg[counter]
                data[l][properties[4]] = mst[counter]
                data[l][properties[5]] = pos[counter]
                data[l][properties[6]]
                data[l][properties[7]]

            pickle.dump(data,open('Main_Data/'+ str(i) + '/' + str(i)+'_data_dic.pkl','wb'))

    return  



#create a descriptor and add into pkl thing  
#n.b. adds two dic from magrescap
def descriptor(rcut, nmax, lmax, sigma):
    species = ['Si', 'O']
    rbf = 'polynomial'
    soap = SOAP(species=species, rbf=rbf, rcut=rcut, nmax=nmax, lmax=lmax, sigma=sigma)

    uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists

    keys = list(uid_dict.keys())
    keys.remove('1')

    for i in keys:
        if os.path.exists('Main_Data/' + str(i) + '/' + str(i) + '.magres'): #maybe useful check elsewhere errorwise
            magres = ase.io.read('Main_Data/' + str(i) + '/' + str(i) + '.magres')
            desc = soap.create(magres)

            data = pickle.load(open('Main_Data/'+ str(i) + '/' + str(i)+'_data_dic.pkl','rb')) #maybe check this exists too
            counter = 0
            for l in data.keys():
                data[l]['descriptor'] = desc[counter]
                counter += 1
            
            pickle.dump(data,open('Main_Data/'+ str(i) + '/' + str(i)+'_data_dic.pkl','wb'))
    
    return


#creates the soap kernel value for ML
#appends into data dic
def soap_kerneler(ref_uid = 10, ref_atom = 'Si_1', ker_exp = 10):
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        return  print('no dic found')
    
    keys = list(uid_dict.keys())
    keys.remove('1')

    if os.path.exists('Main_Data/' + str(ref_uid) + '/' + str(ref_uid) + '.magres'):
        ref_dic = pickle.load(open('Main_Data/' + str(ref_uid) + '/' + str(ref_uid) + '_data_dic.pkl','rb'))
        if ref_dic[ref_atom]['descriptor'] is None:
            return print('no descriptor found for ref atom')
        else:
            ref = ref_dic[ref_atom]['descriptor']
    else:
        return print('no data dic found for uid ref')
    
    for i in keys:
        if os.path.exists('Main_Data/' + str(i) + '/' + str(i)+ '_data_dic.pkl'):
            data = pickle.load(open('Main_Data/' + str(i) + '/' + str(i)+ '_data_dic.pkl', 'rb'))
            for l in data:
                if data[l]['descriptor'] is None:
                    print('no descriptor value for ' + str(i))
                    break
                else:
                    p = data[l]['descriptor']
                    k = (np.dot(p,ref)/(np.dot(p,p)*np.dot(ref,ref))**0.5)**ker_exp
                    data[l]['kernel'] = k

            pickle.dump(data,open('Main_Data/'+ str(i) + '/' + str(i)+'_data_dic.pkl','wb'))
    return
        
                

    


    
#merge all indiviual data dics for each structure into a master.  
#n.b. will overwrite file if no new file name defined
def master_merger(file_out='master_data_dic.pkl'):
    #check if UID dic exists and open
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        return  print('no dic found')
    keys = list(uid_dict.keys())
    keys.remove('1')
    
    to_merge = [] #could also manually choose which to join
    for i in keys:
        if os.path.exists('Main_Data/' + str(i) + '/' + str(i)+ '_data_dic.pkl'):
            to_merge.append(i)
    
    master_dic = {}       
    for i in to_merge:
        minor_dic = pickle.load(open('Main_Data/' + str(i) + '/' + str(i)+ '_data_dic.pkl', 'rb'))
        master_dic[i] = minor_dic

    pickle.dump(master_dic, open(file_out,'wb'))
    return master_dic
    
                
 #NEEDED


  #MAYBE USEFUL

#load a pickle object
def loader():
    return
#dump a pickle object
def dumper():
    return
#check if a dic exists options for what to do e.g. skip/overwrite
def dic_check():
    return



  #OLD -- DONT USE
    
#capture data from all magres files
#update to export to pickle object
#need options to append previous or create new etc
#as above would need to integrate with dates implemented

###This makes one master dic### DONT USE
def magres_cap_old():
    data = {}
    properties = ['iso','vzz','efg','fefg','mst']
    
    if os.path.exists('uid_index.pkl'):#check if exists
        uid_dict = pickle.load(open('uid_index.pkl','rb')) # load dic from pickle if exists
    else:
        return  print('no dic found')
    
    keys = list(uid_dict.keys())
    keys.remove('1')
    
    for i in keys:
        counter = -1

        magres = ase.io.read('Magres/' + str(i) + '.magres')
        atom_spec = magres.get_array('labels')
        indices = magres.get_array('indices')
        labels = ["{0}_{1}".format(l, i) for l, i in zip(atom_spec, indices)]

        iso = MSIsotropy.get(magres)
        vzz = EFGVzz.get(magres)
        efg = EFGAnisotropy.get(magres)
        fefg = magres.get_array('efg')
        mst = magres.get_array('ms')


        #creating a empty nested dictioary to store data
        data[i] = {}
        data[i] = dict.fromkeys(labels)

        for l in labels:
            counter += 1

            data[i][l] = dict.fromkeys(properties)

            data[i][l][properties[0]] = iso[counter]
            data[i][l][properties[1]] = vzz[counter]
            data[i][l][properties[2]] = efg[counter]
            data[i][l][properties[3]] = fefg[counter]
            data[i][l][properties[4]] = mst[counter]
        
    return data


#cif2cell
#need to add a check for if cif2cell actually works or errors out
def cif2cell():
    raw_cifs = glob.glob('Cifs/Raw_Cifs/*.cif')
    
    for i in raw_cifs:
        #shutil.copyfile(i, 'Cifs/Backup/' + i[14:]) #copys file to backup
        
        subprocess.run('cif2cell ' + i + ' --program=castep -o ' + i[:-4] + '.cell', shell=True) #run cif2cell
        
        shutil.move(i, 'Cifs/Backup/')
        if os.path.exists(i[:-4] + '.cell'):
            shutil.move(i[:-4] + '.cell', 'Cells/Raw Cells/') #move to raw cells file
        else:
            print('no .cell found for ' + i)

In [5]:
#Create all inputs and test castep run
ciffer()
cell_index()
dir_maker()
cell_corrector(0.06)
param_writer(700,'PBE')
dry_runner()


#castep_runner(2,4)
#magres_copyer()
#magres_cap()
#descriptor(2.5, 10, 9, 0.2)
#soap_kerneler
#master_merger() #n.b define file name or will use defaults

{2: {'O_1': {'iso': 265.33804046678813,
   'vzz': 0.9276129892545097,
   'efg': 1.391419483881764,
   'fefg': array([[-0.15811863,  0.53623666,  0.15976008],
          [ 0.53623666,  0.53020787,  0.33240241],
          [ 0.15976008,  0.33240241, -0.37208924]]),
   'mst': array([[257.89084132,  30.50826021,  10.77957208],
          [ 27.94918186, 291.31771234,  16.86337403],
          [ 11.53058321,  17.59418752, 246.80556775]]),
   'position': array([0., 0., 0.]),
   'descriptor': array([ 1.08876722e-07,  2.24526621e-06, -1.33885936e-06, ...,
           4.89048564e-08, -3.67501069e-09,  2.76162830e-10]),
   'kernel': 2.255336988149325e-47},
  'O_2': {'iso': 265.33804046678813,
   'vzz': 0.9276129892545104,
   'efg': 1.3914194838817662,
   'fefg': array([[-0.15811863, -0.53623666,  0.15976008],
          [-0.53623666,  0.53020787, -0.33240241],
          [ 0.15976008, -0.33240241, -0.37208924]]),
   'mst': array([[257.89084132, -30.50826021,  10.77957208],
          [-27.94918186, 291.3